# **🧠 Chain Of Thoughts Using GPT-4o**

In [ ]:
!pip install openai
!pip install PyPDF2

In [13]:
import torch
import PyPDF2
import re
import time
from openai import OpenAI

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Checking if CUDA Connected Successfully
print(device)

In [24]:
KEY_PATH = '/content/key.txt'

with open(KEY_PATH, 'r') as file:
    text = file.read()

GPT_API_KEY = f"{text}"

In [17]:
PDF_PATH = '/content/calculus_contents.pdf'

In [18]:
# Function to remove URLs from the text
def remove_links(text):
    # Regex to match URLs (http, https, www)
    url_pattern = r'http[s]?://\S+|www\.\S+'
    clean_text = re.sub(url_pattern, '', text)
    return clean_text

# Function to read the PDF file and extract text without links
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in range(len(reader.pages)):
            page_text = reader.pages[page].extract_text()

            # Remove links from each page's text
            text += remove_links(page_text)
        return text


# Extract the text from the PDF
pdf_content = read_pdf(PDF_PATH)

# print(pdf_content)

In [19]:
n_problems = 10
n_distractors = 3

problems_mode = 'hard'  # Easy - Medium - Hard

In [20]:
responses = ['', '', '', '']

def getPrompt(responses, n):

  if   (n == 1):
    return f"Generate {n_problems} Math problems based on the content I will give you, don't give answers or distractors, the content: {pdf_content}"

  elif (n == 2):
    return f"Answer this {n_problems} math problems with the final answer only: {responses[0]}"

  elif (n == 3):
    return f"In this {n_problems} math answers generate {n_distractors} tricky distractors so I can use them to trick the students in MCQ questions, Answers: {responses[1]}"

  elif (n == 4):
    return f"Merge this {n_problems} math problems and this {n_distractors + 1} possible answers to use them as {n_problems} MCQ exam, The problems: {responses[0]}, The possible answers: {responses[2]}"

In [21]:
client = OpenAI(
    api_key = GPT_API_KEY
)

In [25]:
MODEL = "gpt-4o"
MAX_TOKENS = 1000
TEMP = 0.2

for i in range(0, 4):
  completion = client.chat.completions.create(
      model = MODEL,
      # max_tokens = MAX_TOKENS,
      temperature = TEMP,

      messages=[
          {
              "role" : "system",
              "content" : f"You are an expert AI assistant with advanced reasoning capabilities in mathematics. Your task is to generate {problems_mode} multiple choice math exam to students to solve it"
          },

          {
              "role" : "user",
              "content" : getPrompt(responses, (i + 1))
          },

          {
              "role" : "assistant",
              "content" : "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."
          }
      ]
  )
  responses[i] = completion.choices[0].message.content
  time.sleep(0.1)

In [26]:
print(responses[3])

### Calculus Exam: Multiple Choice Questions

1. **Differentiation of Trigonometric Functions**
   What is the derivative of \( f(x) = \sec^{-1}(x) \)?
   - A) \( \frac{1}{|x|\sqrt{x^2 - 1}} \)
   - B) \( \frac{1}{x\sqrt{x^2 - 1}} \)
   - C) \( \frac{1}{\sqrt{x^2 - 1}} \)
   - D) \( \frac{1}{|x|} \)

2. **Implicit Differentiation**
   For the equation \( x^2 + y^2 = 25 \), what is \( \frac{dy}{dx} \)?
   - A) \( -\frac{x}{y} \)
   - B) \( \frac{y}{x} \)
   - C) \( \frac{x}{y} \)
   - D) \( -\frac{y}{x} \)

3. **Parametric Differentiation**
   For the parametric equations \( x = t^2 + 1 \) and \( y = t^3 - t \), what is \( \frac{dy}{dx} \)?
   - A) \( \frac{3t^2 - 1}{2t} \)
   - B) \( \frac{3t^2 + 1}{2t} \)
   - C) \( \frac{3t - 1}{2t} \)
   - D) \( \frac{3t^2 - 1}{t} \)

4. **Higher Derivatives**
   What is the second derivative \( \frac{d^2y}{dx^2} \) for the function \( y = e^{3x} \sin(x) \)?
   - A) \( 2e^{3x} \cos(x) \)
   - B) \( 3e^{3x} \cos(x) \)
   - C) \( 2e^{3x} \sin(x) \)
  